In [34]:
# Imports
import numpy as np
import cv2
from sklearn import preprocessing
import skimage
from instaFilters import gotham, gingham, clarendon, juno, lark, reyes

# return scene probability vector
# suppose we have x supported scenes
NUM_SCENES = 9
FEATURE_LENGTH = 30
def scene_vector(image):
    scene_prob = np.zeros(NUM_SCENES)
    scene_prob[0] = 1
    return scene_prob[..., np.newaxis] # NUM_SCENES x 1

from skimage.color import rgb2hsv
# normalized feature vector
# expects float32 image
def get_features(image):
    # just a color histogram for now (remember to test hsv)
    # https://docs.opencv.org/3.1.0/d1/db7/tutorial_py_histogram_begins.html
    # remember mask param
    image = rgb2hsv(image).astype('float32')
    hist_1 = cv2.calcHist([image], [0], None, [20], [0, 1])
    hist_2 = cv2.calcHist([image], [1], None, [5], [0, 1])
    hist_3 = cv2.calcHist([image], [2], None, [5], [0, 1])
    return np.concatenate((hist_1, hist_2, hist_3))

# return dictionary of 'avg' histogram/feature vector of scenes given images (key = scenes)
# this should probably be some NN
def scene_features(images):
    scene_features = np.zeros((NUM_SCENES, FEATURE_LENGTH)) 
    for image in images:
        scene = scene_vector(image)
        feature = get_features(image.astype('float32'))
        img_hist = np.matmul(scene, feature.T) # image's contributions to scene features - ith row is ith scene's hist
        scene_features += img_hist
        # matrix multiply scene and feature
        # each image contributes normalized amount of info
    normal = preprocessing.normalize(scene_features)
    return preprocessing.normalize(scene_features)



In [35]:
# return array of distribution vectors for each filter fn in FILTERS
# Todo - add gotham back
FILTERS = [gotham, gingham, clarendon, juno, lark, reyes]
SCENES = ['abbey', 'airport_terminal', 'amphitheater', 'amusement_park', 'aquarium', 'aqueduct', 'art_gallery', 'assembly_line', 'auditorium']
CORPUS_CHARACTERISTIC = './corpus_characteristics.npy'
DATA_DIR = 'data/images/train/a/abbey'
def create_scene_characteristics():
    import glob
    filelist = glob.glob('{}/*.jpg'.format(DATA_DIR))
    images = np.array([cv2.imread(fname) for fname in filelist])
    images = np.asarray([skimage.img_as_float(image).astype('float32') for image in images])
    # write characteristics to file
    filtered_images = [images]
    for filt in FILTERS:
        filtered_images.append(filt(images))
    feature_distributions = []
    for images in filtered_images:
        feature_distributions.append(scene_features(images))
    all_characteristics = np.asarray(feature_distributions)
    np.save(CORPUS_CHARACTERISTIC, all_characteristics)
    # shape is (filters + 1) * (scenes) * (feature length)
def load_scene_characteristics():
    return np.load(CORPUS_CHARACTERISTIC)

create_scene_characteristics()

(1000, 128, 128, 3)


/home/joel/anaconda2/envs/pyEnv/lib/python3.5/site-packages/skimage/color/colorconv.py:278: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 2. + (arr[idx, 2] - arr[idx, 0]) / delta[idx]
/home/joel/anaconda2/envs/pyEnv/lib/python3.5/site-packages/skimage/color/colorconv.py:282: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 4. + (arr[idx, 0] - arr[idx, 1]) / delta[idx]
/home/joel/anaconda2/envs/pyEnv/lib/python3.5/site-packages/skimage/color/colorconv.py:274: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = (arr[idx, 1] - arr[idx, 2]) / delta[idx]
/home/joel/anaconda2/envs/pyEnv/lib/python3.5/site-packages/skimage/color/colorconv.py:268: RuntimeWarning: divide by zero encountered in true_divide
  out_s = delta / out_v


In [38]:
# Ported from stack overflow
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def classifyFilter(img, characteristics):
    scene_vec = scene_vector(img)[0]
#     print(scene_vec)
    # Take dominant scene for now and reference just that
    scene = np.argmax(scene_vec)
    scene_filter_features = characteristics[:, scene]
    img_features = get_features(skimage.img_as_float(img).astype('float32'))
    error_vec = scene_filter_features - img_features.T
#     print(scene_filter_features.shape)
#     print(img_features.T.shape)
#     print(error_vec.shape)
#     print(error_vec)
    errors = np.linalg.norm(error_vec, axis=1)
    print(errors)
    scores = 1 / errors
#     print(scores)
    return softmax(scores)
    
def generate_test_images(fn):
    image = cv2.imread(fn)
    fn_base = fn.split('.')[0]
    import imageio
    imageio.imwrite('data/tests/{}_base.jpg'.format(fn_base), image)
    print(image.shape)
    image = skimage.img_as_float(image).astype('float32')
    from skimage import img_as_ubyte
    for i, filt in enumerate(FILTERS):
        imageio.imwrite('data/tests/{}_{}.jpg'.format(fn_base, i), img_as_ubyte(filt(image)))
# generate_test_images('abbey_test.jpg')
    
def driver():
    img = cv2.imread('abbey_test.jpg')
    characteristics = load_scene_characteristics()
    x = classifyFilter(img, characteristics)
    print(x)

In [39]:
driver()

[ 13696.74111356  13696.80175937  13696.9228207   13696.70299518
  13696.757963    13696.72947437  13696.87611617]
[ 0.14285714  0.14285714  0.14285714  0.14285714  0.14285714  0.14285714
  0.14285714]
